In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer 
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold 

In [ ]:
nltk.download('stopwords')

In [3]:
#Carregando o dataset FAKEBR

dados = pd.read_csv('dataset_shuffle.csv')
X = dados['text']
y = dados['fake_news']
X.head()

0    Internauta filma homem flutuando entre raios. ...
1    Nós cometemos erros, diz Mark Zuckerberg sobre...
2    Brasileiro (que mora nos EUA) compara Miami co...
3    Mídia internacional destaca o vidente Carlinho...
4    FIOCRUZ alerta que vírus zika pode ser transmi...
Name: text, dtype: object

In [4]:
stop_por = set(stopwords.words("portuguese"))
def limpa_texto(string):
    text = string.lower().split()
    text = " ".join(text)
    text = re.sub(r"http(\S)+",' ',text)    
    text = re.sub(r"www(\S)+",' ',text)
    text = re.sub(r"&",' and ',text)  
    text = re.sub('&amp',' ', text)
    text = re.sub(" \d+", " ", text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.split()
    text = [w for w in text if not w in stop_por]
    text = " ".join(text)
    return text

In [5]:
X = X.map(lambda x: limpa_texto(x))

In [6]:
# CountVectorizer / TfidfTransformer Pode ser resumido nisto: 
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
#basta enviar todos os seus documentos aqui
fitted_vectorizer=tfidf_vectorizer.fit(X)
tfidf_vectorizer_vectors=fitted_vectorizer.transform(X)

# Testes com dataset COVID19BR - LinearSVC

In [11]:
X_ = tfidf_vectorizer_vectors

#Implementando a validação cruzada
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = LinearSVC()

#lista das matrizes de confusão de cada iteração 
lista_conf_matrix = []
lista_acc_score = []
lista_precision_score = []
lista_recall_score = []
lista_f1_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X_[train_index],X_[test_index]
    y_train , y_test = y[train_index],y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    #print(confusion_matrix(y_test, pred_values))
    conf_matrix = confusion_matrix(y_test, pred_values)
    lista_conf_matrix.append(conf_matrix)
    
    acc = accuracy_score(pred_values, y_test)    
    lista_acc_score.append(acc)

    prec = precision_score(pred_values, y_test, average = 'weighted', labels=np.unique(y_test))
    lista_precision_score.append(prec)
    
    recall = recall_score(pred_values, y_test, average = 'weighted')
    lista_recall_score.append(recall)
    
    f1 = f1_score(pred_values, y_test, average = 'weighted')     
    lista_f1_score.append(f1)
    
media_conf_matrix = np.mean(lista_conf_matrix, axis=0)
media_acc_score = sum(lista_acc_score)/k
media_prec_score = sum(lista_precision_score)/k
media_recall_score = sum(lista_recall_score)/k
media_f1_score = sum(lista_f1_score)/k

print('Matriz de confusão:')
print(media_conf_matrix)
#print('Acurácia de cada iteração : {}'.format(acc_score))
print('Acurácia média: {}'.format(media_acc_score))
print('Precisão média: {}'.format(media_prec_score))
print('Recall médio  : {}'.format(media_recall_score))
print('F1 médio      : {}'.format(media_f1_score))     

Matriz de confusão:
[[673.4  46.6]
 [ 25.6 694.4]]
Acurácia média: 0.9498611111111112
Precisão média: 0.9526289246683657
Recall médio  : 0.9498611111111112
F1 médio      : 0.9455733519664935


# Testes com dataset COVID19BR - Regressão Logística

In [12]:
X_ = tfidf_vectorizer_vectors

#Implementando a validação cruzada
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = LogisticRegression(solver= 'liblinear')
 
#lista das matrizes de confusão de cada iteração 
lista_conf_matrix = []
lista_acc_score = []
lista_precision_score = []
lista_recall_score = []
lista_f1_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X_[train_index],X_[test_index]
    y_train , y_test = y[train_index],y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    #print(confusion_matrix(y_test, pred_values))
    conf_matrix = confusion_matrix(y_test, pred_values)
    lista_conf_matrix.append(conf_matrix)
    
    acc = accuracy_score(pred_values, y_test)    
    lista_acc_score.append(acc)

    prec = precision_score(pred_values, y_test, average = 'weighted', labels=np.unique(y_test))
    lista_precision_score.append(prec)
    
    recall = recall_score(pred_values, y_test, average = 'weighted')
    lista_recall_score.append(recall)
    
    f1 = f1_score(pred_values, y_test, average = 'weighted')     
    lista_f1_score.append(f1)
    
media_conf_matrix = np.mean(lista_conf_matrix, axis=0)
media_acc_score = sum(lista_acc_score)/k
media_prec_score = sum(lista_precision_score)/k
media_recall_score = sum(lista_recall_score)/k
media_f1_score = sum(lista_f1_score)/k

print('Matriz de confusão:')
print(media_conf_matrix)
#print('Acurácia de cada iteração : {}'.format(acc_score))
print('Acurácia média: {}'.format(media_acc_score))
print('Precisão média: {}'.format(media_prec_score))
print('Recall médio  : {}'.format(media_recall_score))
print('F1 médio      : {}'.format(media_f1_score))     

Matriz de confusão:
[[646.2  73.8]
 [ 30.2 689.8]]
Acurácia média: 0.9277777777777778
Precisão média: 0.9345354677146049
Recall médio  : 0.9277777777777778
F1 médio      : 0.9191487606323298


# Testes com dataset COVID19BR - GradientBoostingClassifier

In [13]:
X_ = tfidf_vectorizer_vectors

#Implementando a validação cruzada
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = GradientBoostingClassifier()

#lista das matrizes de confusão de cada iteração 
lista_conf_matrix = []
lista_acc_score = []
lista_precision_score = []
lista_recall_score = []
lista_f1_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X_[train_index],X_[test_index]
    y_train , y_test = y[train_index],y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    #print(confusion_matrix(y_test, pred_values))
    conf_matrix = confusion_matrix(y_test, pred_values)
    lista_conf_matrix.append(conf_matrix)
    
    acc = accuracy_score(pred_values, y_test)    
    lista_acc_score.append(acc)

    prec = precision_score(pred_values, y_test, average = 'weighted', labels=np.unique(y_test))
    lista_precision_score.append(prec)
    
    recall = recall_score(pred_values, y_test, average = 'weighted')
    lista_recall_score.append(recall)
    
    f1 = f1_score(pred_values, y_test, average = 'weighted')     
    lista_f1_score.append(f1)
    
media_conf_matrix = np.mean(lista_conf_matrix, axis=0)
media_acc_score = sum(lista_acc_score)/k
media_prec_score = sum(lista_precision_score)/k
media_recall_score = sum(lista_recall_score)/k
media_f1_score = sum(lista_f1_score)/k

print('Matriz de confusão:')
print(media_conf_matrix)
#print('Acurácia de cada iteração : {}'.format(acc_score))
print('Acurácia média: {}'.format(media_acc_score))
print('Precisão média: {}'.format(media_prec_score))
print('Recall médio  : {}'.format(media_recall_score))
print('F1 médio      : {}'.format(media_f1_score))     

Matriz de confusão:
[[663.4  56.6]
 [ 32.  688. ]]
Acurácia média: 0.9384722222222223
Precisão média: 0.943059165546192
Recall médio  : 0.9384722222222223
F1 médio      : 0.9336701904821535


# Testes com dataset COVID19BR - Árvore de decisão

In [14]:
X_ = tfidf_vectorizer_vectors

#Implementando a validação cruzada
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = tree.DecisionTreeClassifier()

#lista das matrizes de confusão de cada iteração 
lista_conf_matrix = []
lista_acc_score = []
lista_precision_score = []
lista_recall_score = []
lista_f1_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X_[train_index],X_[test_index]
    y_train , y_test = y[train_index],y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    #print(confusion_matrix(y_test, pred_values))
    conf_matrix = confusion_matrix(y_test, pred_values)
    lista_conf_matrix.append(conf_matrix)
    
    acc = accuracy_score(pred_values, y_test)    
    lista_acc_score.append(acc)

    prec = precision_score(pred_values, y_test, average = 'weighted', labels=np.unique(y_test))
    lista_precision_score.append(prec)
    
    recall = recall_score(pred_values, y_test, average = 'weighted')
    lista_recall_score.append(recall)
    
    f1 = f1_score(pred_values, y_test, average = 'weighted')     
    lista_f1_score.append(f1)
    
media_conf_matrix = np.mean(lista_conf_matrix, axis=0)
media_acc_score = sum(lista_acc_score)/k
media_prec_score = sum(lista_precision_score)/k
media_recall_score = sum(lista_recall_score)/k
media_f1_score = sum(lista_f1_score)/k

print('Matriz de confusão:')
print(media_conf_matrix)
#print('Acurácia de cada iteração : {}'.format(acc_score))
print('Acurácia média: {}'.format(media_acc_score))
print('Precisão média: {}'.format(media_prec_score))
print('Recall médio  : {}'.format(media_recall_score))
print('F1 médio      : {}'.format(media_f1_score))     

Matriz de confusão:
[[625.4  94.6]
 [ 83.2 636.8]]
Acurácia média: 0.8765277777777778
Precisão média: 0.8797176980440481
Recall médio  : 0.8765277777777778
F1 médio      : 0.8617099875428643


# Testes com o dataset COVID19BR - Rede Neural MLP

In [15]:
X_ = tfidf_vectorizer_vectors

#Implementando a validação cruzada
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = MLPClassifier()

#lista das matrizes de confusão de cada iteração 
lista_conf_matrix = []
lista_acc_score = []
lista_precision_score = []
lista_recall_score = []
lista_f1_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X_[train_index],X_[test_index]
    y_train , y_test = y[train_index],y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    #print(confusion_matrix(y_test, pred_values))
    conf_matrix = confusion_matrix(y_test, pred_values)
    lista_conf_matrix.append(conf_matrix)
    
    acc = accuracy_score(pred_values, y_test)    
    lista_acc_score.append(acc)

    prec = precision_score(pred_values, y_test, average = 'weighted', labels=np.unique(y_test))
    lista_precision_score.append(prec)
    
    recall = recall_score(pred_values, y_test, average = 'weighted')
    lista_recall_score.append(recall)
    
    f1 = f1_score(pred_values, y_test, average = 'weighted')     
    lista_f1_score.append(f1)
    
media_conf_matrix = np.mean(lista_conf_matrix, axis=0)
media_acc_score = sum(lista_acc_score)/k
media_prec_score = sum(lista_precision_score)/k
media_recall_score = sum(lista_recall_score)/k
media_f1_score = sum(lista_f1_score)/k

print('Matriz de confusão:')
print(media_conf_matrix)
#print('Acurácia de cada iteração : {}'.format(acc_score))
print('Acurácia média: {}'.format(media_acc_score))
print('Precisão média: {}'.format(media_prec_score))
print('Recall médio  : {}'.format(media_recall_score))
print('F1 médio      : {}'.format(media_f1_score))     

Matriz de confusão:
[[682.8  37.2]
 [ 40.6 679.4]]
Acurácia média: 0.9459722222222222
Precisão média: 0.9476804394099986
Recall médio  : 0.9459722222222222
F1 médio      : 0.9394268605419811
